# Employee Sales Forecasting - EDA & Visual Analysis
## Comprehensive Exploratory Data Analysis

**Objectives:**
- Understand sales patterns and distributions
- Identify seasonality and trends
- Analyze employee performance drivers
- Discover correlations and relationships
- Validate business assumptions

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Styling
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("Set2")

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 2)

print("✅ Libraries loaded successfully!")

## 1. Load Data

In [ ]:
# Load the generated data
df = pd.read_csv('../data/raw/employee_sales_data.csv')
df['date'] = pd.to_datetime(df['date'])

print(f"📊 Dataset Shape: {df.shape}")
print(f"📅 Date Range: {df['date'].min()} to {df['date'].max()}")
print(f"👥 Number of Employees: {df['employee_id'].nunique()}")
print(f"\n📋 Columns: {list(df.columns)}")

## 2. Sales Distribution Analysis

In [ ]:
# Comprehensive sales statistics
print("📈 SALES STATISTICS")
print("=" * 60)
print(f"Mean Sales: ${df['sales'].mean():,.2f}")
print(f"Median Sales: ${df['sales'].median():,.2f}")
print(f"Std Dev: ${df['sales'].std():,.2f}")
print(f"Min Sales: ${df['sales'].min():,.2f}")
print(f"Max Sales: ${df['sales'].max():,.2f}")
print(f"Total Sales: ${df['sales'].sum():,.2f}")
print(f"\nCoefficient of Variation: {(df['sales'].std() / df['sales'].mean() * 100):.2f}%")

# Quartiles
print(f"\n📊 Quartiles:")
print(f"Q1 (25%): ${df['sales'].quantile(0.25):,.2f}")
print(f"Q2 (50%): ${df['sales'].quantile(0.50):,.2f}")
print(f"Q3 (75%): ${df['sales'].quantile(0.75):,.2f}")

In [ ]:
# Visual distribution analysis
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
fig.suptitle('Sales Distribution Analysis', fontsize=16, fontweight='bold', y=1.00)

# 1. Histogram
axes[0, 0].hist(df['sales'], bins=60, edgecolor='black', alpha=0.7, color='steelblue')
axes[0, 0].axvline(df['sales'].mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: ${df["sales"].mean():,.0f}')
axes[0, 0].axvline(df['sales'].median(), color='green', linestyle='--', linewidth=2, label=f'Median: ${df["sales"].median():,.0f}')
axes[0, 0].set_title('Sales Distribution', fontweight='bold')
axes[0, 0].set_xlabel('Sales ($)')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# 2. Box plot
axes[0, 1].boxplot(df['sales'], vert=True, patch_artist=True,
                   boxprops=dict(facecolor='lightblue', color='navy'),
                   medianprops=dict(color='red', linewidth=2))
axes[0, 1].set_title('Sales Box Plot', fontweight='bold')
axes[0, 1].set_ylabel('Sales ($)')
axes[0, 1].grid(True, alpha=0.3)

# 3. Q-Q plot for normality
stats.probplot(df['sales'], dist="norm", plot=axes[0, 2])
axes[0, 2].set_title('Q-Q Plot (Normality Check)', fontweight='bold')
axes[0, 2].grid(True, alpha=0.3)

# 4. Sales by Month
monthly_avg = df.groupby('month')['sales'].mean()
axes[1, 0].bar(monthly_avg.index, monthly_avg.values, color='coral', edgecolor='black')
axes[1, 0].set_title('Average Sales by Month', fontweight='bold')
axes[1, 0].set_xlabel('Month')
axes[1, 0].set_ylabel('Average Sales ($)')
axes[1, 0].set_xticks(range(1, 13))
axes[1, 0].grid(True, alpha=0.3, axis='y')

# 5. Sales by Quarter
quarterly_avg = df.groupby('quarter')['sales'].mean()
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#FFA07A']
axes[1, 1].bar(quarterly_avg.index, quarterly_avg.values, color=colors, edgecolor='black')
axes[1, 1].set_title('Average Sales by Quarter', fontweight='bold')
axes[1, 1].set_xlabel('Quarter')
axes[1, 1].set_ylabel('Average Sales ($)')
axes[1, 1].set_xticks([1, 2, 3, 4])
axes[1, 1].grid(True, alpha=0.3, axis='y')

# 6. Holiday vs Non-Holiday
holiday_comparison = df.groupby('is_holiday_season')['sales'].mean()
axes[1, 2].bar(['Non-Holiday', 'Holiday Season'], holiday_comparison.values, 
               color=['lightblue', 'gold'], edgecolor='black')
axes[1, 2].set_title('Sales: Holiday vs Non-Holiday', fontweight='bold')
axes[1, 2].set_ylabel('Average Sales ($)')
axes[1, 2].grid(True, alpha=0.3, axis='y')

# Add percentage increase
increase = (holiday_comparison[1] / holiday_comparison[0] - 1) * 100
axes[1, 2].text(1, holiday_comparison[1] + 2000, f'+{increase:.1f}%', 
                ha='center', fontweight='bold', fontsize=12, color='darkgreen')

plt.tight_layout()
plt.savefig('../reports/figures/02_sales_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\n✅ Holiday Season Boost: +{increase:.1f}%")

## 3. Time Series Analysis

In [ ]:
# Aggregate sales over time
time_series = df.groupby('date').agg({
    'sales': ['sum', 'mean', 'std', 'count']
}).reset_index()
time_series.columns = ['date', 'total_sales', 'avg_sales', 'std_sales', 'count']

# Calculate rolling statistics
time_series['rolling_mean_3'] = time_series['total_sales'].rolling(window=3).mean()
time_series['rolling_mean_6'] = time_series['total_sales'].rolling(window=6).mean()

# Create comprehensive time series plot
fig, axes = plt.subplots(3, 1, figsize=(16, 12))
fig.suptitle('Time Series Analysis - Sales Trends', fontsize=16, fontweight='bold')

# 1. Total Sales Over Time with Rolling Averages
axes[0].plot(time_series['date'], time_series['total_sales'], 
             label='Total Sales', linewidth=1.5, alpha=0.7, color='steelblue')
axes[0].plot(time_series['date'], time_series['rolling_mean_3'], 
             label='3-Month MA', linewidth=2, color='red')
axes[0].plot(time_series['date'], time_series['rolling_mean_6'], 
             label='6-Month MA', linewidth=2, color='green')
axes[0].set_title('Total Monthly Sales with Moving Averages', fontweight='bold', fontsize=12)
axes[0].set_xlabel('Date')
axes[0].set_ylabel('Total Sales ($)')
axes[0].legend(loc='best')
axes[0].grid(True, alpha=0.3)
axes[0].tick_params(axis='x', rotation=45)

# 2. Average Sales per Employee
axes[1].plot(time_series['date'], time_series['avg_sales'], 
             linewidth=2, color='coral', marker='o', markersize=4)
axes[1].fill_between(time_series['date'], 
                      time_series['avg_sales'] - time_series['std_sales'],
                      time_series['avg_sales'] + time_series['std_sales'],
                      alpha=0.3, color='coral')
axes[1].set_title('Average Sales per Employee (with ±1 Std Dev)', fontweight='bold', fontsize=12)
axes[1].set_xlabel('Date')
axes[1].set_ylabel('Average Sales ($)')
axes[1].grid(True, alpha=0.3)
axes[1].tick_params(axis='x', rotation=45)

# 3. Year-over-Year Comparison
df['year_month'] = df['date'].dt.to_period('M')
yoy_data = df.groupby(['year', 'month'])['sales'].mean().reset_index()
for year in yoy_data['year'].unique():
    year_data = yoy_data[yoy_data['year'] == year]
    axes[2].plot(year_data['month'], year_data['sales'], 
                 marker='o', linewidth=2, label=f'Year {year}')
axes[2].set_title('Year-over-Year Monthly Sales Comparison', fontweight='bold', fontsize=12)
axes[2].set_xlabel('Month')
axes[2].set_ylabel('Average Sales ($)')
axes[2].set_xticks(range(1, 13))
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../reports/figures/03_time_series_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Time series analysis complete!")

## 4. Employee Performance Analysis

In [ ]:
# Employee-level aggregations
employee_stats = df.groupby('employee_id').agg({
    'sales': ['mean', 'std', 'min', 'max', 'sum'],
    'performance_score': 'mean',
    'experience_years': 'first',
    'region': 'first',
    'department': 'first',
    'deals_closed': 'sum'
}).reset_index()

employee_stats.columns = ['employee_id', 'avg_sales', 'std_sales', 'min_sales', 
                          'max_sales', 'total_sales', 'avg_performance', 
                          'experience_years', 'region', 'department', 'total_deals']

# Categorize employees
employee_stats['performance_category'] = pd.qcut(employee_stats['avg_sales'], 
                                                  q=4, 
                                                  labels=['Low', 'Medium', 'High', 'Top'])

print("👥 EMPLOYEE PERFORMANCE DISTRIBUTION")
print("=" * 60)
print(employee_stats['performance_category'].value_counts().sort_index())
print(f"\nTop 10 Performers:")
print(employee_stats.nlargest(10, 'avg_sales')[['employee_id', 'avg_sales', 'total_deals', 'avg_performance']])

In [ ]:
# Employee performance visualization
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
fig.suptitle('Employee Performance Analysis', fontsize=16, fontweight='bold')

# 1. Performance distribution
axes[0, 0].hist(employee_stats['avg_sales'], bins=30, edgecolor='black', color='skyblue')
axes[0, 0].axvline(employee_stats['avg_sales'].mean(), color='red', 
                   linestyle='--', linewidth=2, label='Mean')
axes[0, 0].set_title('Distribution of Avg Employee Sales', fontweight='bold')
axes[0, 0].set_xlabel('Average Sales ($)')
axes[0, 0].set_ylabel('Number of Employees')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# 2. Sales by Region
region_stats = employee_stats.groupby('region')['avg_sales'].agg(['mean', 'std']).sort_values('mean')
axes[0, 1].barh(region_stats.index, region_stats['mean'], 
                xerr=region_stats['std'], color='lightgreen', edgecolor='black')
axes[0, 1].set_title('Average Sales by Region (±Std Dev)', fontweight='bold')
axes[0, 1].set_xlabel('Average Sales ($)')
axes[0, 1].grid(True, alpha=0.3)

# 3. Sales by Department
dept_stats = employee_stats.groupby('department')['avg_sales'].agg(['mean', 'std']).sort_values('mean')
axes[0, 2].barh(dept_stats.index, dept_stats['mean'], 
                xerr=dept_stats['std'], color='lightcoral', edgecolor='black')
axes[0, 2].set_title('Average Sales by Department (±Std Dev)', fontweight='bold')
axes[0, 2].set_xlabel('Average Sales ($)')
axes[0, 2].grid(True, alpha=0.3)

# 4. Experience vs Sales
axes[1, 0].scatter(employee_stats['experience_years'], employee_stats['avg_sales'], 
                   alpha=0.6, s=100, c=employee_stats['avg_performance'], cmap='viridis')
axes[1, 0].set_title('Experience vs Sales', fontweight='bold')
axes[1, 0].set_xlabel('Experience (years)')
axes[1, 0].set_ylabel('Average Sales ($)')
axes[1, 0].grid(True, alpha=0.3)

# Add correlation
corr = employee_stats['experience_years'].corr(employee_stats['avg_sales'])
axes[1, 0].text(0.05, 0.95, f'Correlation: {corr:.3f}', 
                transform=axes[1, 0].transAxes, fontweight='bold',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# 5. Performance Score vs Sales
axes[1, 1].scatter(employee_stats['avg_performance'], employee_stats['avg_sales'], 
                   alpha=0.6, s=100, color='purple')
axes[1, 1].set_title('Performance Score vs Sales', fontweight='bold')
axes[1, 1].set_xlabel('Average Performance Score')
axes[1, 1].set_ylabel('Average Sales ($)')
axes[1, 1].grid(True, alpha=0.3)

# Add correlation
corr2 = employee_stats['avg_performance'].corr(employee_stats['avg_sales'])
axes[1, 1].text(0.05, 0.95, f'Correlation: {corr2:.3f}', 
                transform=axes[1, 1].transAxes, fontweight='bold',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# 6. Performance Categories
cat_counts = employee_stats['performance_category'].value_counts().sort_index()
colors = ['#FF6B6B', '#FFA07A', '#98D8C8', '#6BCF7F']
axes[1, 2].pie(cat_counts.values, labels=cat_counts.index, autopct='%1.1f%%',
               colors=colors, startangle=90)
axes[1, 2].set_title('Employee Performance Categories', fontweight='bold')

plt.tight_layout()
plt.savefig('../reports/figures/04_employee_performance.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Employee analysis complete!")

## 5. Correlation Analysis

In [ ]:
# Select numerical features for correlation
correlation_features = ['sales', 'month', 'quarter', 'is_holiday_season', 
                        'training_hours', 'performance_score', 'deals_closed',
                        'customer_meetings', 'experience_years', 'territory_size',
                        'market_potential', 'competition_level']

corr_matrix = df[correlation_features].corr()

# Create correlation heatmap
plt.figure(figsize=(14, 10))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Feature Correlation Matrix', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.savefig('../reports/figures/05_correlation_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

# Print top correlations with sales
sales_corr = corr_matrix['sales'].sort_values(ascending=False)
print("\n📊 TOP CORRELATIONS WITH SALES:")
print("=" * 60)
print(sales_corr)

## 6. Interactive Plotly Visualizations

In [ ]:
# Interactive time series
fig = px.line(time_series, x='date', y='total_sales', 
              title='Interactive Total Sales Over Time',
              labels={'total_sales': 'Total Sales ($)', 'date': 'Date'})
fig.update_traces(line_color='#1f77b4', line_width=2)
fig.update_layout(hovermode='x unified', height=500)
fig.show()

# Save as HTML
fig.write_html('../reports/figures/06_interactive_timeseries.html')
print("✅ Interactive plot saved!")

In [ ]:
# Regional performance comparison
region_monthly = df.groupby(['date', 'region'])['sales'].sum().reset_index()

fig = px.line(region_monthly, x='date', y='sales', color='region',
              title='Sales Performance by Region Over Time',
              labels={'sales': 'Total Sales ($)', 'date': 'Date', 'region': 'Region'})
fig.update_layout(hovermode='x unified', height=500)
fig.show()

fig.write_html('../reports/figures/07_regional_comparison.html')
print("✅ Regional comparison saved!")

## 7. Key Insights Summary

In [ ]:
print("\n" + "="*70)
print("🎯 KEY INSIGHTS FROM EDA")
print("="*70)

# 1. Seasonality
holiday_boost = (df[df['is_holiday_season']==1]['sales'].mean() / 
                 df[df['is_holiday_season']==0]['sales'].mean() - 1) * 100
print(f"\n📈 1. SEASONALITY")
print(f"   • Holiday season (Nov-Dec) shows {holiday_boost:.1f}% sales increase")
print(f"   • Q4 is consistently the strongest quarter")

# 2. Employee Performance
top_20_pct = employee_stats.nlargest(int(len(employee_stats)*0.2), 'total_sales')['total_sales'].sum()
total_sales = employee_stats['total_sales'].sum()
print(f"\n👥 2. EMPLOYEE PERFORMANCE")
print(f"   • Top 20% of employees generate {(top_20_pct/total_sales*100):.1f}% of total sales")
print(f"   • Performance score correlation with sales: {corr2:.3f}")
print(f"   • Experience correlation with sales: {corr:.3f}")

# 3. Regional Insights
best_region = employee_stats.groupby('region')['avg_sales'].mean().idxmax()
worst_region = employee_stats.groupby('region')['avg_sales'].mean().idxmin()
print(f"\n🌍 3. REGIONAL INSIGHTS")
print(f"   • Best performing region: {best_region}")
print(f"   • Lowest performing region: {worst_region}")

# 4. Growth Trend
early_avg = df[df['date'] < '2022-07-01']['sales'].mean()
late_avg = df[df['date'] >= '2024-01-01']['sales'].mean()
growth = (late_avg / early_avg - 1) * 100
print(f"\n📊 4. GROWTH TRENDS")
print(f"   • Overall sales growth: {growth:.1f}%")
print(f"   • Consistent upward trend observed")

# 5. Variability
cv = (df['sales'].std() / df['sales'].mean()) * 100
print(f"\n📉 5. VARIABILITY")
print(f"   • Sales coefficient of variation: {cv:.1f}%")
print(f"   • Moderate variability suggests predictable patterns")

print("\n" + "="*70)
print("✅ EDA COMPLETE - Ready for Model Development")
print("="*70)